# Language detection: Spacy vs Pycld2

This notebook will compare the language detection between Spacy and Pycld2

In [16]:
import pandas as pd
from colorama import Fore

In [17]:
import spacy
from spacy_langdetect import LanguageDetector

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")
# ca_fasttext_wiki
# es_core_news_sm
# en_core_web_sm

nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [18]:
import pycld2 as cld2

In [19]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/"
debug = 1

In [20]:
def load_data():
    languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_sm", 
                 "en":"en_core_web_sm", "xx":"" }

    data_total = pd.DataFrame(columns=["Comentari","TipusIncidencia","Idioma"])

    for language in languages:
        model = languages[language]
        file = "comentaris_" + language + ".csv"
        if (debug >= 1):
            print ("language: ", language)
            print ("model: ", model)
            print ("file:", file)

        # Load data from file

        data_csv = pd.read_csv(pathdest + file)
        data_lang = data_csv[["Comentari","TipusIncidencia","IdiomaSpacy","IdiomaPyCld2","Idioma"]]
        if (debug >= 2):
            display (data_lang.sample(5))
        if (debug >= 1):
            print ("rows:", data_lang.shape[0])
            print ("")

        data_total = pd.concat([data_total, data_lang])
        
    data_total = data_total[data_total["Comentari"].notnull()]
    data_total = data_total.reset_index()
    if (debug >= 1):
        print ("data_total: ", data_total.shape[0], " rows")

    if (debug >= 2):
        display (data_total.sample(5))
        
    return data_total

In [21]:
data = load_data()
if (debug >= 1):
    display(data)

language:  ca
model:  ca_fasttext_wiki
file: comentaris_ca.csv
rows: 6199

language:  es
model:  es_core_news_sm
file: comentaris_es.csv
rows: 1370

language:  en
model:  en_core_web_sm
file: comentaris_en.csv
rows: 232

language:  xx
model:  
file: comentaris_xx.csv
rows: 91

data_total:  7892  rows


,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
0,0,Els continguts d'aquesta assignatura ja els ha...,NaN,ca,ca,ca
1,1,L'actitud de la professora em resulta molt mot...,NaN,ca,ca,pt
2,2,Materials molt interessants i bones orientacio...,NaN,ca,ca,ca
3,3,Ha estat una exigència molt forta de treball: ...,NaN,ca,ca,ca
4,4,Molta satisfacció amb la transparència i l'ant...,NaN,ca,ca,ca
...,...,...,...,...,...,...
7887,86,No he après res.,NaN,fr,fr,fr
7888,87,POC PROFESSIONAL,NaN,pl,pl,ie
7889,88,Un exemple de bon professor,NaN,fr,fr,en
7890,89,PROFE TOP,NaN,vi,vi,en


In [22]:
#   Distribution by Idioma        
data_group = data[["IdiomaSpacy","Comentari"]].groupby(["IdiomaSpacy"]).count()
if debug:
    print ("Idioma Spacy:")
    display (data_group.sort_values("Comentari", ascending=False).head(10))

Idioma Spacy:


,Comentari
IdiomaSpacy,
ca,6020
es,1354
en,322
fr,43
it,41
pt,37
de,11
UNKNOWN,10
nl,8


In [24]:
#   Distribution by IdiomaPycld2        
data_group = data[["IdiomaPyCld2","Comentari"]].groupby(["IdiomaPyCld2"]).count()
if debug:
    print ("Idioma Pycld2:")
    display (data_group.sort_values("Comentari", ascending=False).head(10))

Idioma Pycld2:


,Comentari
IdiomaPyCld2,
ca,5982
es,1347
en,428
fy,25
tk,14
un,13
hu,13
oc,8
pt,7


In [26]:
#   Comparativa idiomas        
data_group = data[["IdiomaSpacy","IdiomaPyCld2","Comentari"]].groupby(["IdiomaSpacy","IdiomaPyCld2"]).count().reset_index()
display (data_group.sort_values("Comentari", ascending=False).head(10))
data_pivot = data_group.pivot(index="IdiomaSpacy", columns="IdiomaPyCld2", values="Comentari")
if debug:
    display (data_pivot)

,IdiomaSpacy,IdiomaPyCld2,Comentari
3,ca,ca,5839
34,es,es,1300
25,en,en,232
5,ca,en,140
24,en,ca,57
53,it,ca,24
31,es,ca,20
33,es,en,20
74,pt,es,19
41,fr,ca,16


IdiomaPyCld2,af,ca,crs,cs,cy,de,en,es,fi,fr,...,lb,ms,oc,pt,sk,tk,tn,un,xh,yo
IdiomaSpacy,,,,,,,,,,,,,,,,,,,,,
UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN
af,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ca,NaN,5839.0,NaN,NaN,NaN,1.0,140.0,11.0,NaN,1.0,...,1.0,1.0,3.0,6.0,1.0,NaN,NaN,2.0,NaN,1.0
de,NaN,4.0,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
en,1.0,57.0,NaN,NaN,NaN,NaN,232.0,7.0,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN
es,NaN,20.0,NaN,1.0,NaN,NaN,20.0,1300.0,NaN,NaN,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
fr,NaN,16.0,NaN,NaN,NaN,NaN,12.0,3.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
hr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hu,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
data_dif = data[data["IdiomaSpacy"] != data["IdiomaPyCld2"]]
display (data_dif)
print (data_dif.shape[0])

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
1,1,L'actitud de la professora em resulta molt mot...,NaN,ca,ca,pt
9,9,Gran profesor,NaN,ca,en,fy
35,35,No l'hem tingut,No ha impartit classe a aquest grup,ca,en,tk
38,38,no l´he tingut,No ha impartit classe a aquest grup,ca,en,ca
40,40,No he tingut aquesta professora,No ha impartit classe a aquest grup,ca,en,ca
...,...,...,...,...,...,...
7886,85,\r\n,NaN,UNKNOWN,UNKNOWN,un
7888,87,POC PROFESSIONAL,NaN,pl,pl,ie
7889,88,Un exemple de bon professor,NaN,fr,fr,en
7890,89,PROFE TOP,NaN,vi,vi,en


517


In [29]:
data_dif = data[(data["IdiomaSpacy"] == "ca") & (data["IdiomaPyCld2"] == "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
85,85,Gemma Filella és una crak,Comentari excel·lent,ca,ca,en
90,90,Gemma Filella és una peça imprescindible per a...,NaN,ca,ca,en
183,183,Algunes classes eren més dinàmiques que altres...,NaN,ca,ca,en
242,242,Bon professor tècnicament parlant.,NaN,ca,ca,en
244,244,Ha estat entretingut.,NaN,ca,ca,en
...,...,...,...,...,...,...
6031,6031,Es inacceptable que una professora exigeigi pu...,NaN,ca,ca,en
6071,6071,"Bona assignatura, útil, flexiblible i s'ha ada...",NaN,ca,ca,en
6078,6078,"Fan falta més professors com ell,que disfrutin...",NaN,ca,ca,en
6095,6095,Res a dir. Perfecte,NaN,ca,ca,en


140  rows


In [30]:
data_dif = data[(data["IdiomaSpacy"] == "ca") & (data["IdiomaPyCld2"] != "ca") & (data["IdiomaPyCld2"] != "es") & (data["IdiomaPyCld2"] != "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
1,1,L'actitud de la professora em resulta molt mot...,NaN,ca,ca,pt
58,58,M'ha sembla correcta.,NaN,ca,ca,ie
68,68,Falten exercicis per fer.,NaN,ca,ca,de
77,77,És un incompetent,NaN,ca,ca,hu
121,121,Per venir a vendre la empresa només...,NaN,ca,ca,fr
219,219,Teoria rapida i temps de practica a classe. Co...,NaN,ca,ca,un
273,273,És un professor correcte.,NaN,ca,ca,hu
400,400,"No puc evaluar-lo, no estava quan va fer classe.",NaN,ca,ca,pt
402,402,Sempre disposat a ajudar en el que necessitis.,NaN,ca,ca,pt
438,438,És una crack!!,NaN,ca,ca,hu


30  rows


In [31]:
data_dif = data[(data["IdiomaSpacy"] == "es") & (data["IdiomaPyCld2"] == "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
6226,27,En los videos tutoriales no hablar tan deprisa.,NaN,es,es,en
6261,62,"comprensible, entendor, bon professor.",NaN,es,es,en
6281,82,las diapositivas deberian ser en castellano y ...,NaN,es,es,en
6288,89,"Parts poc interrelacionades, sense fil conduct...",NaN,es,es,en
6349,150,Profesora creativa y intelectual de todo signi...,NaN,es,es,en
6477,278,Un maestro!,NaN,es,es,en
6502,303,- Muy disponible y atento.\n,NaN,es,es,en
6510,311,Se echan en falta temarios en audio o audiovis...,NaN,es,es,en
6565,366,Su entusiamo es envidiable.,NaN,es,es,en
6716,517,Se agradece su entusiasmo y actitud a la hora ...,NaN,es,es,en


20  rows


In [32]:
data_dif = data[(data["IdiomaSpacy"] == "es") & (data["IdiomaPyCld2"] != "es") & (data["IdiomaPyCld2"] != "ca") 
                & (data["IdiomaPyCld2"] != "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
6223,24,Materiales estupendos para disfrutar y aprender,NaN,es,es,id
6286,87,No em crida l'atenció,NaN,es,es,pt
6335,136,Excelente docente.,NaN,es,es,cs
6377,178,No vimos a en clase a este docente,NaN,es,es,gl
6394,195,Excelente profesor.,NaN,es,es,fy
6470,271,No vimos clase con él,No ha impartit classe a aquest grup,es,es,gl
6471,272,No vimos clase con ella,No ha impartit classe a aquest grup,es,es,gl
6547,348,Excelente para la formación de Master,NaN,es,es,gl
6557,358,mala organización de la asignatura,NaN,es,es,sk
6621,422,Los testes de teoria son tediosos y dificiles.,NaN,es,es,oc


14  rows


In [33]:
data_dif = data[(data["IdiomaSpacy"] == "en") & (data["IdiomaPyCld2"] == "es")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
6247,48,"AL PINCIPIO DESCORDINACIÓN DE LA ASIGNATURA , ...",NaN,es,en,es
6351,152,Un buen profesional,NaN,es,en,es
6658,459,Muy buen profesor!!,NaN,es,en,es
6782,583,Muy buen profe!,NaN,es,en,es
7016,817,"ME HA GUSTADO MUCHO LA PROFESORA, SOBRE TODO, ...",NaN,es,en,es
7022,823,BUEN PROFESOR...HA DESPERTADO INTERÉS EN LA AS...,NaN,es,en,es
7386,1187,NO HA FACILITADO NADA LA SITUACIÓN ACTUAL CON ...,NaN,es,en,es


7  rows


In [34]:
data_dif = data[(data["IdiomaSpacy"] == "en") & (data["IdiomaPyCld2"] != "en") & (data["IdiomaPyCld2"] != "ca")
                & (data["IdiomaPyCld2"] != "es")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
9,9,Gran profesor,NaN,ca,en,fy
35,35,No l'hem tingut,No ha impartit classe a aquest grup,ca,en,tk
82,82,No l’hem tingut,No ha impartit classe a aquest grup,ca,en,tk
95,95,No l'hem tingut,No ha impartit classe a aquest grup,ca,en,tk
99,99,No l’hem tingut,No ha impartit classe a aquest grup,ca,en,tk
128,128,És mlt bon professor ;),NaN,ca,en,hu
359,359,No l'hem tingut,No ha impartit classe a aquest grup,ca,en,tk
378,378,No l’hem tingut,NaN,ca,en,tk
444,444,No l’hem tingut,NaN,ca,en,tk
771,771,No l'hem tingut.,No ha impartit classe a aquest grup,ca,en,tk


26  rows


In [35]:
data_dif = data[(data["IdiomaSpacy"] != "ca") & (data["IdiomaSpacy"] != "es") & (data["IdiomaSpacy"] != "en")
                & (data["IdiomaPyCld2"] == "ca")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
129,129,No ens expliqueu coses que no ens interessen,NaN,ca,fr,ca
140,140,AMB AQUESTA ASSIGNATURA S'HA POGUT OBTINDRE UN...,NaN,ca,de,ca
203,203,Bona gent,NaN,ca,nl,ca
204,204,Molt bona gent,NaN,ca,nl,ca
243,243,"Ben fet, m'ha agradat.",NaN,ca,sv,ca
...,...,...,...,...,...,...
5595,5595,Molt comprensiva.,NaN,ca,it,ca
5622,5622,Assignatura molt interessant!,NaN,ca,it,ca
5877,5877,REMARCAR LA VOCACIÓ DELS DOS PROFESSORS,NaN,ca,de,ca
6008,6008,Les professores a vegades et diuen coses contr...,NaN,ca,fr,ca


66  rows


In [36]:
data_dif = data[(data["IdiomaSpacy"] != "ca") & (data["IdiomaSpacy"] != "es") & (data["IdiomaSpacy"] != "en")
                & (data["IdiomaPyCld2"] != "ca") & (data["IdiomaPyCld2"] != "es") & (data["IdiomaPyCld2"] != "en")]
display (data_dif)
print (data_dif.shape[0], " rows")

,index,Comentari,TipusIncidencia,Idioma,IdiomaSpacy,IdiomaPyCld2
1230,1230,Quan he tingut algun problema o dubte m'ha int...,NaN,NaN,NaN,NaN
1231,1231,ca,ca,NaN,NaN,NaN
2373,2373,Molts cops fa comentaris que estan fora de llo...,Comentari problemàtic,NaN,NaN,NaN
2374,2374,ca,ca,NaN,NaN,NaN
3282,3282,Pel que fa a la part de l'assignatura de la Do...,Comentari de professor,NaN,NaN,NaN
...,...,...,...,...,...,...
7881,80,-,NaN,UNKNOWN,UNKNOWN,un
7885,84,\r\n,NaN,UNKNOWN,UNKNOWN,un
7886,85,\r\n,NaN,UNKNOWN,UNKNOWN,un
7887,86,No he après res.,NaN,fr,fr,fr


65  rows


In [37]:
# Registro con idioma diferente agrupados por tipo de incidencia

data_dif = data[data["IdiomaSpacy"] != data["IdiomaPyCld2"]]
data_incid = data_dif[data_dif["TipusIncidencia"].notnull()]
data_group = data_incid[["TipusIncidencia","Comentari"]].groupby(["TipusIncidencia"]).count()
data_group

,Comentari
TipusIncidencia,
Comentari de professor,2
Comentari excel·lent,1
Comentari ofensiu,4
Comentari problemàtic,7
Exclamacions o emoticones excessius,1
No ha impartit classe a aquest grup,59
ca,5


Nivel general de coincidencia:
- Textos con igual idioma: 7.351 / 7855 = 93,58 %
- Textos con idioma diferente: 504 registros. Candidatos a recuperarse.

Revisión idioma catalán:
- Los textos detectados ca (spacy) - en (pycld2) són en realidad en catalán (141 casos)
- Los textos detectados ca (spacy) - es (pycld2) són en realidad en catalán (11 casos)
- Los textos detectados ca (spacy) - xx (pycld2) són en realidad en catalán (31 casos)
Conclusión: En general la detección de catalán de Spacy es correcta

Revisión idioma castellano:
- Los textos detectados es (spacy) - ca (pycld2) són en realidad en catalán (16 casos)
- Los textos detectados es (spacy) - en (pycld2) són en realidad en castellano (19 casos)
- Los textos detectados es (spacy) - xx (pycld2) són en realidad en castellano (13 casos)
Conclusión: En geneal la detección del castellano en Spacy es correcta, salvo para el idioma catalán, que se podria mejorar mediante pycld2 

Revisión idioma inglés:
- Los textos detectados en (spacy) - ca (pycld2) són en realidad en catalán (60 casos)
- Los textos detectados en (spacy) - es (pycld2) són en realidad en castellano (8 casos)
- Los textos detectados en (spacy) - xx (pycld2) són en realidad en catalán (25 casos)
Conclusión: Para el castellano y el catalán la detección de pycld2 es mejor y en el caso de otras lenguas se debería hacer una asignación directa al catalán.

Revisión otras lenguas (xx representa las lengua que no son ca, es, en):
- Los textos detectados xx (spacy) - ca (pycld2) són en realidad en catalán (68 casos)
- Los textos detectados xx (spacy) - es (pycld2) són en realidad en castellano (28 casos)
- Los textos detectados xx (spacy) - en (pycld2) són en realidad en catalán (30 casos)
- Los textos detectados xx (spacy) - xx (pycld2) són casi todos catalán y el resto castellano (55 casos)

Conclusión: Para el castellano y el catalán la detección de pycld2 es mejor y en el caso de otras lenguas se debería hacer una asignación directa al catalán.

Agrupación por tipos de incidencia:
- De los 505 registro recuperables 69 son incidencias.
- La mayoría de las incidencias (58 registros) son de tipo 'No ha impartido clase'.
- En todo el conjunto de datos hay 440 incidencias, 37 de tipo 'No ha impartido clase'.


In [14]:
# Detection of language using Spacy and PyCld2 and convination of both results

def detect_language(comment):
    
    lang_spacy = nlp(comment)._.language["language"]
    isReliable, textBytesFound, details = cld2.detect(comment.encode('utf-8', 'replace'), 
                                                    isPlainText = True, bestEffort = True, returnVectors=False)
    lang_pycld2 = details[0][1]
    
    language = lang_spacy
    if (lang_spacy != "ca") & (lang_pycld2 == "ca"):
        language = "ca"

    if (lang_spacy != "ca") & (lang_spacy != "es") & (lang_pycld2 == "es"):
        language = "es"
        
    if (lang_spacy == "en") & (lang_pycld2 != "ca") & (lang_pycld2 != "es") & (lang_pycld2 != "en"):
        language = "ca"
        
    if (debug == 2):
        print (comment, ", ", lang_spacy, ", ", lang_pycld2, ", ", language)
        
    return lang_spacy, lang_pycld2, language

In [15]:
# Mix of languages detection

comments = ["Algunes classes eren més dinàmiques que altres.",
            "El primer dia de classe no es va presentar",
            "Es inacceptable que una professora exigeigi tant",
            "Sempre disposat a ajudar en el que necessitis.",
            "Esta asignatura es una de las más importantes",
            "Alguna pràctica podría estar millor explicada",
            "En los videos tutoriales no hablar tan deprisa.",
            "Materiales estupendos para disfrutar y aprender.",
            "It was a very interactive class and learning.",
            "Criteris poc clars.",
            "Un buen profesional",
            "No l'hem tingut",
            "Assignatura molt interessant!",
            "ME HA GUSTADO MUCHO LA PROFESORA, SOBRE TODO",
            "BUEN PROFESOR...HA DESPERTADO INTERÉS EN LA ASIGNATURA.",
            "bona professora, propera i motivadora 	"]

for comment in comments:
    lang_spacy, lang_pycld2, language = detect_language(comment)
        
    print (comment, ", ", lang_spacy, ", ", lang_pycld2, ", ", language)

Algunes classes eren més dinàmiques que altres. ,  ca ,  ca ,  ca
El primer dia de classe no es va presentar ,  ca ,  es ,  ca
Es inacceptable que una professora exigeigi tant ,  ca ,  en ,  ca
Sempre disposat a ajudar en el que necessitis. ,  ca ,  pt ,  ca
Esta asignatura es una de las más importantes ,  es ,  es ,  es
Alguna pràctica podría estar millor explicada ,  es ,  ca ,  ca
En los videos tutoriales no hablar tan deprisa. ,  es ,  en ,  es
Materiales estupendos para disfrutar y aprender. ,  es ,  id ,  es
It was a very interactive class and learning. ,  en ,  en ,  en
Criteris poc clars. ,  en ,  ca ,  ca
Un buen profesional ,  en ,  es ,  es
No l'hem tingut ,  en ,  tk ,  ca
Assignatura molt interessant! ,  it ,  ca ,  ca
ME HA GUSTADO MUCHO LA PROFESORA, SOBRE TODO ,  pt ,  es ,  es
BUEN PROFESOR...HA DESPERTADO INTERÉS EN LA ASIGNATURA. ,  de ,  en ,  de
bona professora, propera i motivadora 	 ,  it ,  fy ,  it
